# ipyleaflet for CAR maps
We first import all needed material

In [1]:
from ipyleaflet import (
    Map, MapStyle,
    TileLayer, LocalTileLayer, ColorizableTileLayer, Graticule, 
    MapRangeControl, MousePositionControl, FullScreenControl, LayersControl, WidgetControl,
)
import ipywidgets as widgets

The we define the base map that will embed all the different layers 

In [2]:
def deflayer(url, name):
    return ColorizableTileLayer(url="files/" + url,
                                base=True,
                                min_zoom=-5,
                                max_zoom=+5,
                                min_native_zoom=-5,
                                max_native_zoom=0,                            
                                tile_size=675,
                                attribution="",
                                name=name,
                                show_loading=False,
                                colormap="planck")

m = Map(
    layers=(Graticule(),),
    controls=(FullScreenControl(), 
              MousePositionControl(), 
              MapRangeControl(), 
              LayersControl(collapsed=False)),
    crs="CAR",
    center=(0, 0),
#    min_zoom=-10,
#    max_zoom=+10,
    interpolation="nearest",
    zoom=0,
    scroll_wheel_zoom=True,
    fade_animation=False,
    world_copy_jump=True,
    #style=MapStyle(cursor="default"),
    default_style=MapStyle(cursor="default"),
    #dragging_style=MapStyle(cursor="default")
   )

#m.layout.height = '700px'

We can show the map using the `sidecar` plugin for jupyter lab.

In [3]:
from sidecar import Sidecar
sc = Sidecar(title='CMB map')
with sc:
   display(m)

Now we add the different layers corresponding to the different maps

In [4]:
layers = [deflayer("tiles/split%s_IQU.fits/{z}/tile_{y}_{x}_%s.png" % (str(split), str(i)), "CMB - {} - split {}".format(item, split)) 
          for i, item in enumerate("IQU") for split in [0, 1]]
for layer in layers:
    m.add_layer(layer)

We now add widgets to change the colormaps and the color range.

In [5]:
scale = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=10.0,
    step=0.1,
    description="scale",
    continuous_update=False,
    orientation='vertical',
    readout_format='.1f',
    layout=widgets.Layout(width='100px')
)
def on_scale_change(change):
    for layer in layers:
        layer.scale = change["new"]
scale.observe(on_scale_change, names='value')
m.add_control(WidgetControl(widget=scale, position='topright'))

from ipyleaflet import allowed_colormaps
cmap = widgets.RadioButtons(
    options=allowed_colormaps,
    value='planck', 
    description='',
    layout=widgets.Layout(width='100px')
)
def on_cmap_change(change):
    for layer in layers:
        layer.colormap = change["new"]
cmap.observe(on_cmap_change, names='value')
m.add_control(WidgetControl(widget=cmap, position='topright'))

We also add a button widget to start the computation of power spectra if patches are available

In [6]:
button = widgets.Button(description='Process spectra',
                        icon="times", 
                        tooltip="No patches!",
                        disabled=True)
m.add_control(WidgetControl(widget=button, position='bottomright'))

Finally let's add a toolbar to select patches. When a patch is drawn, we store its properties in `json` format

In [7]:
from ipyleaflet import DrawControl
draw_control = DrawControl()
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.circlemarker = {"repeatMode": True}
draw_control.circle = {"repeatMode": True, "metric": False}
draw_control.rectangle = {"repeatMode": True, "showArea": False}

patches = []
def handle_draw(self, action, geo_json):
    if action == "created":
        patches.append(geo_json)
    if len(patches) > 0:
        button.disabled = False
        button.icon = "check"
        button.tooltip = "Click to compute spectra"

draw_control.on_draw(handle_draw)
m.add_control(draw_control)

In [21]:
patches = []

In [8]:
def parse_rectangle(coordinates):
    return [coordinates[0][0][::-1], coordinates[0][2][::-1]]

from psplay.psplay import compute_ps
map0_info = {"name": "psplay/split0_IQU.fits", "data_type": "IQU", "id": "split0", "cal": None}
map1_info = {"name": "psplay/split1_IQU.fits", "data_type": "IQU", "id": "split1", "cal": None}

maps_info_list = [map0_info, map1_info]

# source_mask = {"name": "source_mask.fits", "apo_type": "C1", "apo_radius": 0.3}
# galactic_mask = "mask_galactic_equatorial_car_halfarcmin.fits"

def process_spectra(*args, **kwargs):
    button.icon = 'gear spin lg'
    print("Process spectra")
    for patch in patches:
        geometry = patch.get("geometry")
        if geometry["type"] == "Polygon":
            patch_dict = {
                "patch_type": "Rectangle",
                "patch_coordinate": parse_rectangle(geometry["coordinates"])
            }
            print(patch_dict)
            spectra, spec_name_list, lb, ps, cov = compute_ps(patch_dict,
                                                              maps_info_list,
                                                              ps_method="master",
                                                              error_method_1d="master",
                                                              beam=None,
                                                              binning_file=None,
                                                              bin_size=40,
                                                              type="Dl",
                                                              source_mask=None,
                                                              galactic_mask=None,
                                                              apo_radius_survey=1,
                                                              compute_T_only=False,
                                                              lmax=4000,
                                                              master_threshold=500)
    button.icon = 'check'
    
button.on_click(process_spectra)

ModuleNotFoundError: No module named 'psplay'

In [14]:
button = widgets.Button(description="Running", icon="gear lg")
button

Button(description='Running', icon='gear lg', style=ButtonStyle())

In [11]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
def plot_spectra(spectra, spec_name_list, lb, ps, cov):
    fig, axes = plt.subplots(3, 3, figsize=(15, 12), sharex=True)
    ax = axes.flatten()
    for i, spec in enumerate(spectra):
        for spec_name in spec_name_list:
            ax[i].errorbar(lb, ps[spec_name][spec], np.sqrt(np.diag(cov[spec_name][spec])), fmt=".", label=spec_name)
        ax[i].set_ylabel(r'$D^{%s}_{\ell}$'%spec)

    #fig.legend([], spec_name_list, loc="upper left", bbox_to_anchor=(1,1))

    for ax in axes[-1]:
        ax.set_xlabel(r'$\ell$')
    plt.tight_layout()

In [ ]:
from ipyleaflet import MeasureControl
m.add_control(MeasureControl())

In [ ]:
from ipyleaflet import Marker
mark = Marker(location=m.center)
m += mark